In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [2]:
import pandas as pd
import numpy as np

In [3]:
opts = webdriver.ChromeOptions()
opts.headless =True

In [4]:
path_link = "C:/Users/Sky/.wdm/drivers/chromedriver/win32/101.0.4951.41/chromedriver.exe"

In [5]:
driver = webdriver.Chrome(executable_path = path_link, options = opts)
driver.implicitly_wait(10)

<ipython-input-5-143182bab761>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path_link)


In [6]:
search_url = "https://www.proquest.com/washingtonpost?accountid=29989"
driver.get(search_url)


In [7]:
driver.find_element(By.XPATH, '//*[@id="product-level-nav"]/li[2]').click()

In [8]:
keys = """((federal reserve) OR (fed)) AND((money supply) OR (open market operations) OR
(quantitative easing) OR (monetary policy) OR (fed funds rate) OR (overnight lending rate) OR
Bernanke OR Volker OR Greenspan OR Yellen OR Powell OR (central bank) OR (interest rates) OR (fed chairman) OR
(fed chair) OR (lender of last resort) OR (discount window) OR (European Central Bank) OR ECB OR (Bank of England) OR
(Bank of Japan) OR BOJ OR (Bank of China) OR Bundesbank OR (Bank of France) OR (Bank of Italy))"""

In [10]:
driver.find_element(By.XPATH,'//*[@id="select_multiDateRange"]/option[10]').click()
# Start
driver.find_element(By.XPATH,'//*[@id="month2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="day2"]/option[2]').click()
driver.find_element(By.XPATH,'//*[@id="year2"]').send_keys('2000')
# End
driver.find_element(By.XPATH,'//*[@id="month2_0"]/option[13]').click()
driver.find_element(By.XPATH,'//*[@id="day2_0"]/option[32]').click()
driver.find_element(By.XPATH,'//*[@id="year2_0"]').send_keys('2021')

In [11]:
driver.find_element(By.XPATH,'//*[@id="SourceType_Newspapers"]').click()
driver.find_element(By.XPATH,'//*[@id="Language_ENG"]').click()

In [12]:
driver.find_element(By.XPATH, '//*[@id="queryTermField"]').send_keys(keys)

In [13]:
driver.find_element(By.XPATH, '//*[@id="sortType"]/option[2]').click() # oldest first

In [12]:
driver.find_element(By.XPATH, '//*[@id="itemsPerPage"]/option[4]').click()

In [14]:
dataset = []
pattern_date = '[0-9]{1,2}\s[A-Za-z]{3,5}\s[0-9]{4}'

while True:
    articles = driver.find_elements(By.CLASS_NAME, 'resultItem')

    for article in articles:
        title = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("title")
        date = article.find_element(By.CLASS_NAME,'newspaperArticle').text
        date = re.search(pattern_date, date).group()
    
        link = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("href")
    
        driver_temp = webdriver.Chrome(executable_path = path_link, options = opts)
        driver_temp.implicitly_wait(10)
        driver_temp.get(link)
    
        try:
            full_text = driver_temp.find_element(By.XPATH, '//*[@id="fullTextZone"]/text').text
        except NoSuchElementException:
            full_text = np.nan

        try:
            abstract = driver_temp.find_element(By.CLASS_NAME,'abstract').text
        except NoSuchElementException:
            abstract = np.nan
        
    
        article_info = {'date': date, 'title': title, "abstract": abstract, 'full_text': full_text}
        dataset.append(article_info)
    
        driver_temp.close()
        
    try:
        driver.find_element(By.CSS_SELECTOR, "[title^='Next Page']").click()
        time.sleep(5)
    except NoSuchElementException:
        break

driver.quit()
df = pd.DataFrame(dataset)

<ipython-input-14-327709cc25a4>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_temp = webdriver.Chrome(executable_path = path_link, options = opts)


df

In [15]:
df.head()

,date,title,abstract,full_text
0,01 Jan 2000,The Clintons Host A Historic Fete; A Repast fo...,"Muhammad Ali, Elizabeth Taylor, Sophia Loren, ...",It was a once-in-a lifetime invitation--greeti...
1,01 Jan 2000,Bonds' Bad Year Closes With Whimper: [FINAL Ed...,"""Treasuries are still falling this morning, ta...",While tech stocks boomed and most major equity...
2,01 Jan 2000,Technology-Led Market Keeps Up Momentum: [FINA...,The year's star was the tech-laden Nasdaq comp...,"Stocks ended the year on an up note, capping t..."
3,01 Jan 2000,100 Lives; This Is Who We Were.: [FINAL Edition],My name is Larissa Kosmos. I'm 28. I'm blessed...,My name is Larissa Kosmos. I'm 28. I'm blessed...
4,02 Jan 2000,D.C. Homicides Lowest Since 1987; Killings in ...,Correction: An article Sunday incorrectly repo...,Correction: An article Sunday incorrectly repo...


In [16]:
#df.to_csv("news_data.csv", index = False)